In [1]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
##device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model_name = "google/vit-base-patch16-224-in21k"
#processor = ViTImageProcessor.from_pretrained(model_name)
#feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)


from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification,EfficientNetConfig

model_name="google/efficientnet-b4"
# 1. Load the model and its configuration
processor = EfficientNetImageProcessor.from_pretrained(model_name)
#model = RegNetForImageClassification.from_pretrained("facebook/regnet-y-320-seer-in1k", config=config)



/home/hk6560892/.conda/envs/hk63560892/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-30 00:15:38.054497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 00:15:38.718390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

import copy
from tqdm import tqdm
from torchvision import transforms
import random
from PIL import Image
from torchvision import datasets

import torch.nn as nn
import torchvision.models as models
import torch

import torch.optim as optim


from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


from torchvision import datasets
import gc
import torch
from torch.utils.data import DataLoader, ConcatDataset


# 将权重加载到模型实例中
#model.load_state_dict(model_weights)
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models



import os
import torchvision.transforms as T
from PIL import Image
import torch

# Custom transform to add Gaussian noise
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

# Custom transform to add Speckle noise
class AddSpeckleNoise(object):
    """
    Add speckle noise to the image.
    """
    def __init__(self, noise_level=0.1):
        """
        :param noise_level: Standard deviation of the noise distribution
        """
        self.noise_level = noise_level

    def __call__(self, tensor):
        """
        :param tensor: PyTorch tensor, the image on which noise is added
        :return: PyTorch tensor, image with speckle noise
        """
        # Generate speckle noise
        noise = torch.randn_like(tensor) * self.noise_level

        # Add speckle noise to the image
        noisy_tensor = tensor * (1 + noise)

        # Clip the values to be between 0 and 1
        noisy_tensor = torch.clamp(noisy_tensor, 0, 1)

        return noisy_tensor

class AddPoissonNoise(object):
    """
    Add Poisson noise to the image.
    """
    def __init__(self, lam=1.0):
        """
        :param lam: Lambda parameter for Poisson distribution
        """
        self.lam = lam

    def __call__(self, tensor):
        """
        :param tensor: PyTorch tensor, the image to which noise is added
        :return: PyTorch tensor, image with Poisson noise
        """
        # Generate Poisson noise
        noise = torch.poisson(self.lam * torch.ones(tensor.shape))

        # Add Poisson noise to the image
        noisy_tensor = tensor + noise / 255.0  # Assuming the image is scaled between 0 and 1

        # Clip the values to be between 0 and 1
        noisy_tensor = torch.clamp(noisy_tensor, 0, 1)

        return noisy_tensor

import torch.nn as nn

class Posterize(nn.Module):
    def __init__(self, bits):
        super(Posterize, self).__init__()
        self.bits = bits
        self.shift = 8 - bits

    def forward(self, img):
        # Assuming input is float32 tensor with values in [0, 1]
        # Convert to uint8 in [0, 255]
        img_uint8 = (img * 255).to(torch.uint8)
        
        # Posterize
        img_posterized = img_uint8 >> self.shift
        img_posterized = img_posterized << self.shift
        
        # Convert back to float32 in [0, 1]
        img_float = img_posterized.float() / 255.0
        return img_float

# Custom transform to add Salt and Pepper noise
class AddSaltPepperNoise(object):
    def __init__(self, salt_prob=0.05, pepper_prob=0.05):
        self.salt_prob = salt_prob
        self.pepper_prob = pepper_prob

    def __call__(self, tensor):
        noise = torch.rand(tensor.size())
        tensor[(noise < self.salt_prob)] = 1  # Salt noise: setting some pixels to 1
        tensor[(noise > 1 - self.pepper_prob)] = 0  # Pepper noise: setting some pixels to 0
        return tensor

# Define the image augmentation transformations
aug_transform = T.Compose([
    
    T.Resize(384,interpolation=transforms.InterpolationMode.BICUBIC),
    T.RandomApply([T.RandomHorizontalFlip()], p=0.1),
    T.RandomApply([T.RandomVerticalFlip()], p=0.1),
    T.RandomApply([T.RandomRotation(10)], p=0.1),

    T.RandomApply([T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)], p=0.1),
    T.ToTensor(),  # Convert PIL image to tensor
    T.RandomGrayscale(p=0.1),
    T.RandomInvert(p=0.1),
    #T.RandomPosterize(bits=2, p=0.1),
    T.RandomApply([Posterize(bits=2)], p=0.1),
    T.RandomApply([T.RandomSolarize(threshold=1.0)], p=0.05),
    T.RandomApply([T.RandomAdjustSharpness(sharpness_factor=2)], p=0.1),
    
    
    T.RandomApply([AddGaussianNoise(0., 0.05)], p=0.1),  # mean and std
    T.RandomApply([AddPoissonNoise(lam=0.1)], p=0.1),  # mean and std
    T.RandomApply([AddSpeckleNoise(noise_level=0.1)], p=0.1),
    T.RandomApply([AddSaltPepperNoise(salt_prob=0.05, pepper_prob=0.05)], p=0.1),

    T.RandomApply([T.RandomPerspective(distortion_scale=0.6, p=1.0)], p=0.1),
    T.RandomApply([T.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75))], p=0.1),
    T.RandomApply([T.ElasticTransform(alpha=250.0)], p=0.1),

    T.RandomApply([T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.))], p=0.1),

    T.RandomApply([AddGaussianNoise(0., 0.001)], p=1.0),  # mean and std
    #T.ToPILImage()  # Convert tensor back to PIL image for saving

])
final_transforms = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

full_transform = transforms.Compose([
    aug_transform,  # your existing transformations
    final_transforms
])

In [3]:
def process_example(example):
    inputs = processor(example['image'], return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs


In [4]:
def collate_fn(batch):
    images = [item[0] for item in batch]
    labels = [item[1] for item in batch]

    return {
        'pixel_values': torch.stack(images),
        'labels': torch.tensor(labels)
    }
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


/tmp/ipykernel_18545/689226782.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [5]:
from torch.utils.data import random_split
import gc
from transformers import TrainingArguments
from transformers import Trainer
counter=0
for i in range(4):
    train_dataset = datasets.ImageFolder("train", transform=full_transform)

    train_size = int(0.85 * len(train_dataset))  # let's say you want 80% for training
    val_size = len(train_dataset) - train_size  # remaining for validation/test

    train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
    
    # 获取标签和ID映射
    labels = train_dataset.class_to_idx.items()
    id2label = {str(id): label for label, id in labels}
    label2id = {label: str(id) for label, id in labels}
    from transformers import EfficientNetConfig
    # 從預訓練的配置中加載配置
    # 从预训练模型加载配置
    config = EfficientNetConfig.from_pretrained(model_name)
    # 修改配置
    config.num_labels = len(labels)
    config.id2label = id2label
    config.label2id = label2id

    # 使用該配置初始化模型
    #model = EfficientNetForImageClassification(config)
    model = EfficientNetForImageClassification.from_pretrained(model_name, config=config,ignore_mismatched_sizes=True)
    model.classifier=nn.Linear(in_features=1792, out_features=50, bias=True)




    #model.classifier=nn.Linear(768,50,True)


    for param in model.parameters():
        param.requires_grad = True

    model.num_labels = 50
    #if counter!=0:
    model.load_state_dict(torch.load("effic.pth"))
    #counter+=1


    training_args = TrainingArguments(
        output_dir="./efficientnet-b4-beans",
        per_device_train_batch_size=8,
        evaluation_strategy="steps",  # Change this to "steps"
        num_train_epochs=2,
        fp16=True,
        save_steps=585,  # Save at the end of each epoch
        eval_steps=585,  # Evaluate at the end of each epoch
        logging_steps=10,
        learning_rate=2e-4,
        save_total_limit=20,
        remove_unused_columns=False,
        push_to_hub=False,
        report_to='tensorboard',
        lr_scheduler_type='linear',
        load_best_model_at_end=False,  # Turn this off for now
    )
    

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        train_dataset=train_subset,
        eval_dataset=val_subset,
        tokenizer=processor,
    )
    train_results = trainer.train()
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    torch.save(model.state_dict(), "effic.pth")




Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([50, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([50]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:676.)
  tensor[(noise < self.salt_prob)] = 1  # Salt noise: setting some pixels to 1
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of maske

Step,Training Loss,Validation Loss,Accuracy
585,0.210900,0.326587,0.955299
1170,0.121600,0.216346,0.967334
1755,0.068400,0.330619,0.958944
2340,0.220700,0.374975,0.957843
2925,0.326500,0.345778,0.965614
3510,0.082200,nan,0.972560
4095,0.288800,0.222309,0.974280
4680,0.141400,0.294825,0.969466
5265,0.048800,0.201053,0.975518
5850,0.187300,0.188865,0.974623


/home/hk6560892/.conda/envs/hk63560892/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:676.)
  tensor[(noise < self.salt_prob)] = 1  # Salt noise: setting some pixels to 1
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of masked_fill_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[mask] = scalar (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndex

***** train metrics *****
  epoch                    =          2.0
  total_flos               = 7185179760GF
  train_loss               =       0.1676
  train_runtime            =   3:13:01.13
  train_samples_per_second =        14.23
  train_steps_per_second   =        0.889


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([50, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([50]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

***** train metrics *****
  epoch                    =          2.0
  total_flos               = 7185179760GF
  train_loss               =       0.1526
  train_runtime            =   3:13:40.26
  train_samples_per_second =       14.182
  train_steps_per_second   =        0.886


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b4 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1792]) in the checkpoint and torch.Size([50, 1792]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([50]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy
585,0.022200,0.346355,0.965614
1170,0.273100,0.221350,0.973798
1755,0.051000,0.379197,0.969534
2340,0.094600,0.519576,0.960044
2925,0.187100,0.402881,0.968159
3510,0.014400,nan,0.973661
4095,0.126000,0.338782,0.975586
4680,0.077500,0.313050,0.973386
5265,0.035100,0.285834,0.981638
5850,0.002600,0.325565,0.975242


/home/hk6560892/.conda/envs/hk63560892/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:676.)
  tensor[(noise < self.salt_prob)] = 1  # Salt noise: setting some pixels to 1
/tmp/ipykernel_18545/1417931453.py:130: UserWarning: Use of masked_fill_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[mask] = scalar (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndex

KeyboardInterrupt: 

In [ ]:
from torch.utils.data import random_split
import gc
from transformers import TrainingArguments
from transformers import Trainer
counter=0

train_dataset = datasets.ImageFolder("train", transform=full_transform)

train_size = int(0.85 * len(train_dataset))  # let's say you want 80% for training
val_size = len(train_dataset) - train_size  # remaining for validation/test

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

In [ ]:

model_path = f"./efficientnet-b4-beans/checkpoint-{5265}"


config = EfficientNetConfig.from_pretrained(model_path)
model = EfficientNetForImageClassification.from_pretrained(model_path, config=config)






#model.classifier=nn.Linear(in_features=1792, out_features=50, bias=True)
#model.load_state_dict(torch.load("effic.pth"))
torch.save(model.state_dict(), "effic.pth")

In [ ]:
model

EfficientNetForImageClassification(
  (efficientnet): EfficientNetModel(
    (embeddings): EfficientNetEmbeddings(
      (padding): ZeroPad2d((0, 1, 0, 1))
      (convolution): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=valid, bias=False)
      (batchnorm): BatchNorm2d(48, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (activation): SiLUActivation()
    )
    (encoder): EfficientNetEncoder(
      (blocks): ModuleList(
        (0): EfficientNetBlock(
          (depthwise_conv): EfficientNetDepthwiseLayer(
            (depthwise_conv_pad): ZeroPad2d((0, 1, 0, 1))
            (depthwise_conv): EfficientNetDepthwiseConv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=48, bias=False)
            (depthwise_norm): BatchNorm2d(48, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
            (depthwise_act): SiLUActivation()
          )
          (squeeze_excite): EfficientNetSqueezeExciteLayer(
            (squeeze): Adap

In [ ]:
from torchvision import datasets, transforms
image_transform = transforms.Compose([
    transforms.Resize(384,interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 4. Make sure the model is on the target device
model.to(device)
### Predict on image ### 

# 4. Make sure the model is on the target device
#model.to(device)

# 5. Turn on model evaluation mode and inference mode
model.eval()
from PIL import Image
import os
#pretrained_transform=pretrained_vit_weights.transforms()
# 1. 定义文件夹路径
image_folder_path = "test-final/test-final"

# 1. 定义文件夹路径
image_folder_path = "test-final/test-final"

# 2. 获取文件夹中的所有图像文件名
image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]

# 按照文件名（数字）进行排序
image_files = sorted(image_files, key=lambda x: int(x.split('.')[0]))
import numpy as np

label_index = []
all_pro=[]
# 3. 对每个图像进行预测
for image_file in image_files:
    img_path = os.path.join(image_folder_path, image_file)
    img = Image.open(img_path).convert('RGB')  # 转换为RGB格式

    # 使用您之前的代码进行预测
    #model.to(device)
    model.eval()
    with torch.no_grad():  # 使用torch.no_grad()来关闭梯度计算，这是inference模式的常见做法
        transformed_image = image_transform(img).unsqueeze(dim=0)
        target_image_pred = model(transformed_image.to(device))
        logits = target_image_pred.logits
        target_image_pred_probs = torch.softmax(logits, dim=1)
        all_pro.append(target_image_pred_probs.cpu().numpy())
        target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
        label_index.append(target_image_pred_label.item())

        # 打印预测结果
        print(f"Image {image_file} is predicted as class: {target_image_pred_label.item()}, probability: {max(target_image_pred_probs[0])}")

Image 1.jpg is predicted as class: 11, probability: 0.9999994039535522
Image 2.jpg is predicted as class: 31, probability: 1.0
Image 3.jpg is predicted as class: 47, probability: 1.0
Image 4.jpg is predicted as class: 13, probability: 0.9999991655349731
Image 5.jpg is predicted as class: 18, probability: 0.9999998807907104
Image 6.jpg is predicted as class: 24, probability: 1.0
Image 7.jpg is predicted as class: 41, probability: 1.0
Image 8.jpg is predicted as class: 6, probability: 0.999808132648468
Image 9.jpg is predicted as class: 35, probability: 1.0
Image 10.jpg is predicted as class: 40, probability: 1.0
Image 11.jpg is predicted as class: 32, probability: 0.9999992847442627
Image 12.jpg is predicted as class: 0, probability: 0.9999419450759888
Image 13.jpg is predicted as class: 34, probability: 1.0
Image 14.jpg is predicted as class: 33, probability: 0.9999817609786987
Image 15.jpg is predicted as class: 25, probability: 1.0
Image 16.jpg is predicted as class: 30, probability:

In [ ]:
from torchvision import datasets, transforms
import pandas as pd
train = datasets.ImageFolder(root='train')
# 反转class_to_idx字典以从索引到类名进行查找
idx_to_class = {v: k for k, v in train.class_to_idx.items()}
character_names = [idx_to_class[pred] for pred in label_index]
submission = pd.DataFrame({
    "id": list(range(1, len(character_names)+1)),
    "character": character_names})

# 保存为CSV文件
submission.to_csv("submission.csv", index=False)

In [ ]:
from torchvision import datasets, transforms
image_transform = transforms.Compose([
    transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

### Predict on image ### 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 4. Make sure the model is on the target device
model.to(device)

# 5. Turn on model evaluation mode and inference mode
model.eval()
from PIL import Image
import os
#pretrained_transform=pretrained_vit_weights.transforms()
# 1. 定义文件夹路径
image_folder_path = "test-final/test-final"

# 1. 定义文件夹路径
image_folder_path = "test-final/test-final"

# 2. 获取文件夹中的所有图像文件名
image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]

# 按照文件名（数字）进行排序
image_files = sorted(image_files, key=lambda x: int(x.split('.')[0]))
import numpy as np

label_index = []
all_pro=[]
# 3. 对每个图像进行预测
for image_file in image_files:
    img_path = os.path.join(image_folder_path, image_file)
    img = Image.open(img_path).convert('RGB')  # 转换为RGB格式

    # 使用您之前的代码进行预测
    #model.to(device)
    model.eval()
    with torch.no_grad():  # 使用torch.no_grad()来关闭梯度计算，这是inference模式的常见做法
        transformed_image = image_transform(img).unsqueeze(dim=0)
        target_image_pred = model(transformed_image.to(device))
        logits = target_image_pred.logits
        target_image_pred_probs = torch.softmax(logits, dim=1)
        all_pro.append(target_image_pred_probs.cpu().numpy())
        target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)
        label_index.append(target_image_pred_label.item())

        # 打印预测结果
        print(f"Image {image_file} is predicted as class: {target_image_pred_label.item()}, probability: {max(target_image_pred_probs[0])}")

Image 1.jpg is predicted as class: 11, probability: 0.999555766582489
Image 2.jpg is predicted as class: 31, probability: 0.9997500777244568
Image 3.jpg is predicted as class: 47, probability: 0.9991084933280945
Image 4.jpg is predicted as class: 13, probability: 0.9831324219703674
Image 5.jpg is predicted as class: 18, probability: 0.9997312426567078
Image 6.jpg is predicted as class: 24, probability: 0.9995810389518738
Image 7.jpg is predicted as class: 41, probability: 0.9997956156730652
Image 8.jpg is predicted as class: 47, probability: 0.6951143741607666
Image 9.jpg is predicted as class: 35, probability: 0.9996065497398376
Image 10.jpg is predicted as class: 40, probability: 0.9995726943016052
Image 11.jpg is predicted as class: 32, probability: 0.9993194341659546
Image 12.jpg is predicted as class: 0, probability: 0.9997637867927551
Image 13.jpg is predicted as class: 34, probability: 0.9993389248847961
Image 14.jpg is predicted as class: 33, probability: 0.9997453093528748
Ima

In [ ]:
import numpy as np
import pickle

# 将列表转换为numpy数组
all_probs_array = np.array(all_pro)
#97.219
# 使用pickle将数组保存到文件
with open('all_probs_effic.pkl', 'wb') as f:
    pickle.dump(all_probs_array, f)
#pro1 0.9691
#pro #

In [ ]:
import pickle
with open('all_probs_vit.pkl', 'rb') as f:
    loaded_all_probs = pickle.load(f)


with open('all_probs_focal.pkl', 'rb') as f:
    loaded_all_probs1 = pickle.load(f)

with open('all_probs_con.pkl', 'rb') as f:
    loaded_all_probs2 = pickle.load(f)

with open('all_probs_effic.pkl', 'rb') as f:
    loaded_all_probs3 = pickle.load(f)


import numpy as np

# 確認形狀相同
assert loaded_all_probs.shape == loaded_all_probs1.shape ==loaded_all_probs2.shape,"Shapes of the arrays are not the same!"
loaded_all_probs=loaded_all_probs
loaded_all_probs1=loaded_all_probs1
loaded_all_probs2=loaded_all_probs2
loaded_all_probs3=loaded_all_probs3
# 取得平均機率
average_probs = (loaded_all_probs+loaded_all_probs1+loaded_all_probs2+loaded_all_probs3)/4

In [ ]:
from torchvision import datasets, transforms
import pandas as pd
train = datasets.ImageFolder(root='train')
# 反转class_to_idx字典以从索引到类名进行查找
idx_to_class = {v: k for k, v in train.class_to_idx.items()}
max_prob_indices = np.argmax(average_probs, axis=2)
flattened_indices = max_prob_indices.flatten()
character_names = [idx_to_class[pred] for pred in flattened_indices]
submission = pd.DataFrame({
    "id": list(range(1, len(character_names)+1)),
    "character": character_names})

# 保存为CSV文件
submission.to_csv("submission.csv", index=False)